In [3]:

import pickle
import numpy as np
import os
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx
import onnxruntime as ort

# ============================================
# CONFIGURATION
# ============================================

MODEL_DIR = r"C:\Users\sersi\Desktop\projet_SE_et_IOT\HandSense_project\model_mobile"

model_path = os.path.join(MODEL_DIR, "asl_model_mobile.pkl")
scaler_path = os.path.join(MODEL_DIR, "scaler.pkl")
encoder_path = os.path.join(MODEL_DIR, "label_encoder.pkl")

# ============================================
# FONCTION 1: CONVERTIR EN ONNX
# ============================================

def convert_to_onnx():
    """
    Convertit le modèle sklearn en format ONNX
    """
    print("="*70)
    print("🔄 CONVERSION DU MODÈLE EN ONNX")
    print("="*70)
    
    # Charger le modèle
    print("\n📦 Chargement des fichiers...")
    with open(model_path, "rb") as f:
        model = pickle.load(f)
    print(f"   ✅ Modèle chargé: {model}")
    
    with open(scaler_path, "rb") as f:
        scaler = pickle.load(f)
    print(f"   ✅ Scaler chargé")
    
    with open(encoder_path, "rb") as f:
        label_encoder = pickle.load(f)
    print(f"   ✅ Label Encoder chargé")
    print(f"      Classes: {len(label_encoder.classes_)}")
    
    # Définir l'entrée ONNX
    print(f"\n🔧 Configuration ONNX:")
    print(f"   Shape d'entrée: [None, 11] (batch variable, 11 features)")
    initial_type = [('float_input', FloatTensorType([None, 11]))]
    
    # Convertir
    try:
        print(f"\n⏳ Conversion en cours...")
        onnx_model = convert_sklearn(
            model, 
            initial_types=initial_type,
            target_opset=12,  # Version ONNX compatible
            options={
                'zipmap': False,  # Désactive ZipMap (réduit la taille)
                'nocl': True      # Pas de transformation de classe
            }
        )
        print(f"   ✅ Conversion réussie!")
        
    except MemoryError as e:
        print(f"\n❌ ERREUR: Mémoire insuffisante")
        print(f"   Le modèle est encore trop volumineux.")
        print(f"   Solutions:")
        print(f"   1. Réduire n_estimators à 20")
        print(f"   2. Réduire max_depth à 10")
        print(f"   3. Utiliser TFLite au lieu d'ONNX")
        raise e
    
    # Sauvegarder
    onnx_path = os.path.join(MODEL_DIR, "asl_model.onnx")
    with open(onnx_path, "wb") as f:
        f.write(onnx_model.SerializeToString())
    
    # Taille du fichier
    size_mb = os.path.getsize(onnx_path) / (1024 * 1024)
    print(f"\n💾 Modèle ONNX sauvegardé:")
    print(f"   📁 Chemin: {onnx_path}")
    print(f"   📦 Taille: {size_mb:.2f} MB")
    
    # Vérifier le modèle
    print(f"\n🔍 Vérification du modèle ONNX...")
    try:
        onnx.checker.check_model(onnx_model)
        print(f"   ✅ Modèle ONNX valide!")
    except Exception as e:
        print(f"   ⚠️  Avertissement: {e}")
    
    return onnx_path


# ============================================
# FONCTION 2: TESTER LE MODÈLE ONNX
# ============================================

def test_onnx_model(onnx_path):
    """
    Teste le modèle ONNX avec des données aléatoires
    """
    print("\n" + "="*70)
    print("🧪 TEST DU MODÈLE ONNX")
    print("="*70)
    
    # Charger le label encoder pour afficher les noms
    with open(encoder_path, "rb") as f:
        label_encoder = pickle.load(f)
    
    # Créer une session ONNX Runtime
    print(f"\n⏳ Chargement du modèle ONNX...")
    session = ort.InferenceSession(onnx_path)
    print(f"   ✅ Session ONNX créée")
    
    # Informations sur le modèle
    input_name = session.get_inputs()[0].name
    output_names = [output.name for output in session.get_outputs()]
    
    print(f"\n📊 Informations du modèle:")
    print(f"   Entrée: {input_name}")
    print(f"   Shape: {session.get_inputs()[0].shape}")
    print(f"   Type: {session.get_inputs()[0].type}")
    print(f"   Sorties: {output_names}")
    
    # Créer des données de test (valeurs normalisées)
    print(f"\n🎲 Génération de données de test...")
    test_data = np.random.randn(1, 11).astype(np.float32)  # 1 échantillon, 11 features
    print(f"   Shape: {test_data.shape}")
    print(f"   Données: {test_data[0][:5]}... (premiers 5 valeurs)")
    
    # Prédiction
    print(f"\n⏳ Prédiction en cours...")
    inputs = {input_name: test_data}
    outputs = session.run(output_names, inputs)
    
    # Résultats
    label = outputs[0][0]  # Classe prédite
    probabilities = outputs[1][0]  # Probabilités
    
    predicted_class = label_encoder.classes_[label]
    
    print(f"\n🎯 RÉSULTATS:")
    print(f"   Classe prédite: {predicted_class} (ID: {label})")
    print(f"   Confiance: {probabilities[label]:.4f} ({probabilities[label]*100:.2f}%)")
    
    # Top 3 prédictions
    print(f"\n📊 Top 3 probabilités:")
    top_3_idx = np.argsort(probabilities)[-3:][::-1]
    for i, idx in enumerate(top_3_idx, 1):
        class_name = label_encoder.classes_[idx]
        prob = probabilities[idx]
        print(f"   {i}. {class_name:15s}: {prob:.4f} ({prob*100:.2f}%)")
    
    print(f"\n✅ Test réussi! Le modèle ONNX fonctionne correctement.")


# ============================================
# FONCTION 3: CRÉER UN FICHIER DE TEST POUR REACT NATIVE
# ============================================

def create_test_file():
    """
    Crée un fichier JSON avec des données de test pour React Native
    """
    print("\n" + "="*70)
    print("📝 CRÉATION DU FICHIER DE TEST")
    print("="*70)
    
    # Charger label encoder
    with open(encoder_path, "rb") as f:
        label_encoder = pickle.load(f)
    
    # Créer des données de test
    test_samples = []
    
    # Exemple 1: Geste de repos (tous les capteurs à 0)
    test_samples.append({
        "name": "Rest Position",
        "features": [0.0] * 11,
        "expected_class": "rest"
    })
    
    # Exemple 2: Valeurs aléatoires normalisées
    for i in range(3):
        sample = {
            "name": f"Random Sample {i+1}",
            "features": np.random.randn(11).tolist(),
            "expected_class": "unknown"
        }
        test_samples.append(sample)
    
    # Sauvegarder
    test_file = os.path.join(MODEL_DIR, "test_samples.json")
    import json
    with open(test_file, 'w') as f:
        json.dump({
            "samples": test_samples,
            "classes": label_encoder.classes_.tolist(),
            "n_features": 11,
            "feature_names": [
                'flex_1', 'flex_2', 'flex_3', 'flex_4', 'flex_5',
                'GYRx', 'GYRy', 'GYRz',
                'ACCx', 'ACCy', 'ACCz'
            ]
        }, f, indent=2)
    
    print(f"   ✅ Fichier de test créé: {test_file}")
    print(f"   📊 {len(test_samples)} échantillons de test")


# ============================================
# FONCTION 4: AFFICHER LES INSTRUCTIONS REACT NATIVE
# ============================================

def show_react_native_instructions(onnx_path, label_encoder):
    """
    Affiche les instructions pour intégrer le modèle dans React Native
    """
    print("\n" + "="*70)
    print("📱 INSTRUCTIONS POUR REACT NATIVE")
    print("="*70)
    
    print(f"""
✅ ÉTAPES D'INTÉGRATION:

1️⃣ Installer ONNX Runtime React Native:
   npm install onnxruntime-react-native

2️⃣ Copier les fichiers dans votre projet:
   - Copier {onnx_path} → assets/models/asl_model.onnx
   - Copier {os.path.join(MODEL_DIR, 'test_samples.json')} → assets/test_samples.json

3️⃣ Code exemple React Native:

import {{ InferenceSession }} from 'onnxruntime-react-native';

// Charger le modèle
const session = await InferenceSession.create('asl_model.onnx');

// Préparer les données (11 features normalisées)
const sensorData = [
  flex1, flex2, flex3, flex4, flex5,  // 0-100
  gyrX, gyrY, gyrZ,                    // ±1 rad/s
  accX, accY, accZ                     // ±10 m/s²
];

// Normaliser les données (utiliser les mêmes stats que le scaler)
const normalizedData = new Float32Array(sensorData);

// Prédiction
const feeds = {{ float_input: normalizedData }};
const results = await session.run(feeds);

// Résultats
const label = results.output_label.data[0];
const probabilities = results.output_probability.data;

4️⃣ Classes reconnues:
   {', '.join(label_encoder.classes_.tolist()[:10])}
   ... ({len(label_encoder.classes_)} classes au total)

⚠️  IMPORTANT:
   - Les données doivent être normalisées avant la prédiction
   - Utiliser Float32Array pour les entrées
   - Le modèle attend exactement 11 features

📦 Taille du modèle: {os.path.getsize(onnx_path) / (1024 * 1024):.2f} MB
   Compatible avec Android et iOS!
""")


# ============================================
# FONCTION PRINCIPALE
# ============================================

def main():
    """
    Pipeline complet de conversion
    """
    print("\n" + "="*80)
    print("🚀 CONVERSION DU MODÈLE ASL POUR MOBILE")
    print("="*80)
    
    try:
        # 1. Convertir en ONNX
        onnx_path = convert_to_onnx()
        
        # 2. Tester le modèle
        test_onnx_model(onnx_path)
        
        # 3. Créer fichier de test
        create_test_file()
        
        # 4. Afficher instructions (charger label_encoder ici)
        with open(encoder_path, "rb") as f:
            label_encoder = pickle.load(f)
        show_react_native_instructions(onnx_path, label_encoder)
        
        print("\n" + "="*80)
        print("✨ CONVERSION TERMINÉE AVEC SUCCÈS !")
        print("="*80)
        print(f"\n📁 Fichiers générés dans: {MODEL_DIR}")
        print(f"   ✅ asl_model.onnx")
        print(f"   ✅ test_samples.json")
        print(f"\n🎯 Le modèle est prêt pour React Native!")
        
    except Exception as e:
        print(f"\n❌ ERREUR: {e}")
        print(f"\n💡 Solutions:")
        print(f"   1. Réentraîner avec n_estimators=20")
        print(f"   2. Utiliser TFLite: pip install tensorflow")
        print(f"   3. Vérifier la mémoire disponible")
        raise

if __name__ == "__main__":
    main()


🚀 CONVERSION DU MODÈLE ASL POUR MOBILE
🔄 CONVERSION DU MODÈLE EN ONNX

📦 Chargement des fichiers...
   ✅ Modèle chargé: RandomForestClassifier(max_depth=12, min_samples_leaf=4, min_samples_split=10,
                       n_estimators=30, n_jobs=-1, random_state=42, verbose=1)
   ✅ Scaler chargé
   ✅ Label Encoder chargé
      Classes: 40

🔧 Configuration ONNX:
   Shape d'entrée: [None, 11] (batch variable, 11 features)

⏳ Conversion en cours...
   ✅ Conversion réussie!

💾 Modèle ONNX sauvegardé:
   📁 Chemin: C:\Users\sersi\Desktop\projet_SE_et_IOT\HandSense_project\model_mobile\asl_model.onnx
   📦 Taille: 31.97 MB

🔍 Vérification du modèle ONNX...
   ✅ Modèle ONNX valide!

🧪 TEST DU MODÈLE ONNX

⏳ Chargement du modèle ONNX...
   ✅ Session ONNX créée

📊 Informations du modèle:
   Entrée: float_input
   Shape: [None, 11]
   Type: tensor(float)
   Sorties: ['label', 'probabilities']

🎲 Génération de données de test...
   Shape: (1, 11)
   Données: [-2.112338   -1.3550664   0.32506925 -0